In [232]:
import pickle

#load the data
#this pickle file contains the torsion id: {'molecule':[dihedral indices]}
tid_clusters_list = pickle.load(open('torsion_drive_data.pickle','rb'))
count=0
keys=0

molecule_smiles=[]
for key, item in tid_clusters_list.items():
    print(key)
    keys+=1
    for x, value in item.items():
        print(x)
        print(value)
        molecule_smiles.append(x)
        #print(type(item))
        count+=1
    
print(count)
print(keys)
print(len(set(molecule_smiles)))
with open('molecules.smi', 'w') as f:
    for i in molecule_smiles:
        f.write(i+'\n')

t143f
[H:8][C:1]([H:9])([H:10])[C:2](=[O:3])[N+:4]([H:11])([H:12])[S:5](=[O:6])[O:7][H:13]
[[6, 4, 3, 11]]
[H:15][C:4]1([C:5]([C:6]([N+:2]([C:3]1([H:13])[H:14])([C:1]([H:10])([H:11])[H:12])[S:7](=[O:8])[O:9][H:21])([H:19])[H:20])([H:17])[H:18])[H:16]
[[5, 1, 6, 8]]
[H:8][C:1]([H:9])([H:10])[N+:2]([C:3]([H:11])([H:12])[H:13])([C:4]([H:14])([H:15])[H:16])[S:5](=[O:6])[O:7][H:17]
[[3, 1, 4, 6]]
[H:21][c:7]1[c:6]([c:5]([c:4]([c:9]([c:8]1[H:22])[H:23])[N+:2]([C:1]([H:13])([H:14])[H:15])([C:3]([H:16])([H:17])[H:18])[S:10](=[O:11])[O:12][H:24])[H:19])[H:20]
[[3, 1, 9, 11]]
[H:5][N+:1]([H:6])([H:7])[S:2](=[O:3])[O:4][H:8]
[[3, 1, 0, 6]]
t130b
[H:12][c:1]1[c:2]([c:3]([c:4]([c:5]([c:6]1[H:16])[H:15])[S:7](=[O:8])(=[O:9])[N:10]([H:17])[N:11]([H:18])[H:19])[H:14])[H:13]
[[6, 9, 10, 18]]
[H:19][C:8]([H:20])([H:21])[C:7]([H:17])([H:18])[N:6]([N:5]([H:16])[S:2](=[O:3])(=[O:4])[C:1]([H:13])([H:14])[H:15])[S:9](=[O:10])(=[O:11])[C:12]([H:22])([H:23])[H:24]
[[8, 5, 4, 15]]
[H:20][c:5]1[c:6]([c:7]([c:8](

In [218]:
# copied from https://github.com/openforcefield/qca-dataset-submission/blob/master/submissions/2021-03-23-OpenFF-Amide-Torsion-Set-v1.0/generate-dataset.ipynb
import logging
import warnings
from pprint import pprint

import numpy as np
#from openff.qcsubmit.common_structures import TorsionIndexer
from openff.qcsubmit.workflow_components import TorsionIndexer

from openff.qcsubmit.factories import TorsiondriveDatasetFactory
from openff.toolkit.topology import Molecule

#from openforcefield.topology import Molecule
#from openforcefield.utils import UndefinedStereochemistryError
from openff.toolkit.utils import UndefinedStereochemistryError
from simtk import unit
from tqdm import tqdm
import random

In [219]:
from openff.qcsubmit import common_structures
print(dir(common_structures))

['Any', 'BaseModel', 'ClassVar', 'ClientHandler', 'CommonBase', 'ComponentProperties', 'DatasetConfig', 'DatasetInputError', 'DefaultProperties', 'Dict', 'DriverEnum', 'Enum', 'Field', 'HttpUrl', 'IndexCleaner', 'List', 'Metadata', 'Model', 'Molecule', 'MoleculeAttributes', 'Optional', 'PCMSettingError', 'PCMSettings', 'PositiveInt', 'QCSpec', 'QCSpecificationError', 'QCSpecificationHandler', 'ResultsConfig', 'SCFProperties', 'Set', 'StrictBool', 'StrictFloat', 'StrictInt', 'StrictStr', 'TDSettings', 'Tuple', 'Union', 'WavefunctionProtocolEnum', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'constants', 'constr', 'date', 'datetime', 'getpass', 'np', 'ptl', 're', 'split_openff_molecule', 'validator']


In [220]:
# Warnings that tell us we have undefined stereo and charged molecules
logging.getLogger("openforcefield").setLevel(logging.ERROR)
warnings.simplefilter("ignore")

In [221]:
molecules = []


for key, item in tid_clusters_list.items():
    for mol, indices in item.items():
        smiles_pattern = mol
        #randomly select dihedral indices from the dictionary for the torsion drive
        dihedrals = random.choice(indices)

        # RDKit is unable to understand the stereochemistry of these molecules so
        # we skip them.
        if smiles_pattern == "[H]/C(=C(/C([H])([H])[H])\\[H])/C(=C(\\C([H])([H])[H])/[H])/[H]":
            continue

        try:
            molecule: Molecule = Molecule.from_smiles(smiles_pattern)
        except UndefinedStereochemistryError:
            molecule: Molecule = Molecule.from_smiles(smiles_pattern, allow_undefined_stereo=True)
            molecule = ([molecule] + molecule.enumerate_stereoisomers(max_isomers=1))[-1]

        molecule = molecule.canonical_order_atoms()

        try:
            molecule.generate_conformers(n_conformers=10)
        except Exception:
            print(f"Skipping {smiles_pattern} - OMEGA error.")
            continue
        try:
            molecule.to_rdkit()
        except Exception:
            print(f"Skipping {smiles_pattern} - RDKit incompatible.")
            continue

        torsion_indexer = TorsionIndexer()
        print(tuple(dihedrals))
        #torsion_indexer.add_torsion(tuple(dihedrals), (-165, 180), True)
        torsion_indexer.add_torsion(tuple(dihedrals), (0,3), (-165, 180))

        molecule.properties["dihedrals"] = torsion_indexer
        molecules.append(molecule)

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 16, name: , idx: 9, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 6, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 10, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 11, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 16, name: , idx: 12, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 4, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 13, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 1

(6, 4, 3, 11)


Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 16, name: , idx: 13, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 4, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 14, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 15, aromatic: False, chiral: False



(5, 1, 6, 8)


Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 16, name: , idx: 18, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 9, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 19, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 20, aromatic: False, chiral: False



...

(3, 1, 2, 19)
(3, 1, 2, 16)
(2, 1, 3, 4)
(3, 2, 4, 5)
(3, 2, 4, 5)
(1, 4, 5, 7)
(1, 2, 3, 5)
(3, 1, 4, 7)
(4, 2, 5, 11)
(8, 9, 10, 13)


Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 16, name: , idx: 6, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 3, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 7, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 8, aromatic: False, chiral: False



(4, 1, 0, 5)
(1, 9, 6, 8)
(3, 1, 0, 4)


Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 16, name: , idx: 6, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 2, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 7, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 8, aromatic: False, chiral: False
Problematic bonds are: [<openeye.oechem.OEBondBase; proxy of <Swig Object of type 'OEChem::OEBondBase *' at 0x7f9e38a8d330> >]



(3, 2, 4, 10)
(4, 3, 5, 9)
(4, 3, 5, 6)
(1, 2, 3, 5)


In [222]:
len(molecules)


99

In [223]:
#debug : there is an issue with generating the dataset, 98 molecules are being filtered. 

dataset_factory = TorsiondriveDatasetFactory()


dataset = dataset_factory.create_dataset(
    dataset_name="OpenFF multiplicity correction torsion drive data v1.0",
    tagline="OpenFF multiplicity correction torsion drive data v1.0",
    description="We created new torsion parameters to correct multiplicity issues in the force field. These torsions are for training of the new torsion parameters which do not have preexisting data in the training datasets.",
    molecules=molecules,
)

dataset.metadata.submitter = "jmaat"
dataset.metadata.long_description_url = (
    "https://github.com/openforcefield/qca-dataset-submission/tree/master/"
    "submissions/"
    "2022-04-29-OpenFF-multiplicity-correction-torsion-drive-data-v1.0"
)

Preparation                   :   0%|                    | 0/99 [00:00<?, ?it/s]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 7, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 4, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 8, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 16, name: , idx: 17, aromatic: False, chiral: False

Preparation                   :  14%|█▌         | 14/99 [00:00<00:01, 58.05it/s]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 4, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 1, aromatic: False, chiral: False
bond orde

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 6, name: , idx: 8, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 5, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 9, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 10, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 21, aromatic: False, chiral: False
Atom atomic num: 16, name: , idx: 14, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 13, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 15, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 17, aromatic: False, chiral: False
Proble

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 6, name: , idx: 15, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 12, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 16, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 17, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 21, aromatic: False, chiral: False

Preparation                   : 100%|███████████| 99/99 [00:01<00:00, 65.65it/s]


In [224]:
dir(dataset)
print(list(dataset.molecules))

[Molecule with name '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C:1]([H])([H])[N:3]([H:4])[N:2]([H])[H])[H])[H]' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])C([H])([H])N([H])N([H])[H])[H])[H]']


molecules[12]

In [230]:
confs = np.array([len(mol.conformers) for mol in dataset.molecules])

print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of torsion drives         ", dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in dataset.molecules:
    mass = sum(atom.mass.value_in_unit(unit.dalton) for atom in molecule.atoms)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in dataset.molecules)))

Number of unique molecules        1
Number of filtered molecules      98
Number of torsion drives          1
Number of conformers min mean max 4   4.00 4
Mean molecular weight: 136.20
Max molecular weight: 136.20
Charges: [0.0]


In [226]:
pprint(dataset.metadata.dict())

{'collection_type': 'TorsionDriveDataset',
 'creation_date': datetime.date(2022, 5, 20),
 'dataset_name': 'OpenFF multiplicity correction torsion drive data v1.0',
 'elements': {'C', 'N', 'H'},
 'long_description': 'We created new torsion parameters to correct '
                     'multiplicity issues in the force field. These torsions '
                     'are for training of the new torsion parameters which do '
                     'not have preexisting data in the training datasets.',
 'long_description_url': HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2022-04-29-OpenFF-multiplicity-correction-torsion-drive-data-v1.0', scheme='https', host='github.com', tld='com', host_type='domain', port='443', path='/openforcefield/qca-dataset-submission/tree/master/submissions/2022-04-29-OpenFF-multiplicity-correction-torsion-drive-data-v1.0'),
 'short_description': 'OpenFF multiplicity correction torsion drive data v1.0',
 'submitter': 'jmaat'}


In [227]:
dataset.priority = 'high'

In [228]:
for spec, obj in dataset.qc_specifications.items():
    print("Spec:", spec)
    pprint(obj.dict())

Spec: default
{'basis': 'DZVP',
 'implicit_solvent': None,
 'keywords': None,
 'maxiter': 200,
 'method': 'B3LYP-D3BJ',
 'program': 'psi4',
 'scf_properties': ['dipole',
                    'quadrupole',
                    'wiberg_lowdin_indices',
                    'mayer_indices'],
 'spec_description': 'Standard OpenFF optimization quantum chemistry '
                     'specification.',
 'spec_name': 'default',
 'store_wavefunction': 'none'}


In [229]:
print(dataset.scf_properties)

AttributeError: 'TorsiondriveDataset' object has no attribute 'scf_properties'

In [ ]:
dataset.export_dataset("dataset.json.bz2")
dataset.molecules_to_file("dataset.smi", "smi")

dataset.visualize("dataset.pdf", columns=8)